### Salary prediction, episode II: make it actually work (4 points)

Your main task is to use some of the tricks you've learned on the network and analyze if you can improve __validation MAE__. Try __at least 3 options__ from the list below for a passing grade. Write a short report about what you have tried. More ideas = more bonus points. 

__Please be serious:__ " plot learning curves in MAE/epoch, compare models based on optimal performance, test one change at a time. You know the drill :)

You can use either __pytorch__ or __tensorflow__ or any other framework (e.g. pure __keras__). Feel free to adapt the seminar code for your needs. For tensorflow version, consider `seminar_tf2.ipynb` as a starting point.


# ML utils

In [1]:
%load_ext autoreload

In [54]:
%autoreload 2
from ds_utils.torch_utils import (
    train_val_test_split, device_default,
    Apply, map_idx
)
from ds_utils.train import (
    loopa,
    EarlyStopper,
    mean_metric
)
from ds_utils.plotter import History, Plotter
from ds_utils.magic import reprint

In [3]:
from math import ceil
from dataclasses import dataclass, field
from pathlib import Path
from time import sleep

import pandas as pd
import plotly.express as px
from IPython.display import clear_output
from plotly.subplots import make_subplots
import numpy as np



In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from utils import map, filter
from tqdm.auto import tqdm

%matplotlib inline

In [5]:
# code from seminar

from collections import Counter

import nltk


data = pd.read_csv("./Train_rev1.zip", compression='zip', index_col=None)
data['Log1pSalary'] = np.log1p(data['SalaryNormalized']).astype('float32')

############################################################
text_columns = ["Title", "FullDescription"]
categorical_columns = ["Category", "Company", "LocationNormalized", "ContractType", "ContractTime"]
TARGET_COLUMN = "Log1pSalary"

data[categorical_columns] = data[categorical_columns].fillna('NaN')

############################################################
tokenizer = nltk.tokenize.WordPunctTokenizer()

for col in text_columns:
    data[col] = data[col]\
        .astype(str)\
        .str.lower()\
        .apply(tokenizer.tokenize)\
        .apply(' '.join)


In [6]:
data_train: pd.DataFrame
data_val: pd.DataFrame
data_test: pd.DataFrame

data_train, data_val, data_test  = train_val_test_split(data, val_size=.2, test_size=.1,
                                                        random_state=42)
for df in data_train, data_val, data_test:
    df.reset_index(drop=True, inplace=True)

In [7]:
token_counts = Counter([
    word
    for line in data_train[text_columns].values.flatten()
    for word in line.split()
])

min_count = 10

# tokens from token_counts keys that had at least min_count occurrences throughout the dataset
tokens = sorted(token for token, count in token_counts.items() if count >= min_count)

# Add a special tokens for unknown and empty words
UNK, PAD = "UNK", "PAD"
tokens = [UNK, PAD] + tokens

token_to_id = map_idx(tokens)

In [8]:
import torch
import torch.nn as nn
import torch.functional as F
from sklearn.feature_extraction import DictVectorizer


In [9]:
data_train[categorical_columns].nunique()

Category                 29
Company               17751
LocationNormalized     2436
ContractType              3
ContractTime              3
dtype: int64

In [10]:
data_train["LocationNormalized"].value_counts().value_counts().iloc[:10]

count
1     743
2     259
3     172
4      97
6      91
5      78
7      55
9      53
8      45
11     42
Name: count, dtype: int64

In [11]:
data_train["Company"].value_counts().value_counts().iloc[:10]

count
1     7869
2     2379
3     1289
4      922
5      632
6      534
7      436
8      356
9      323
10     278
Name: count, dtype: int64

In [12]:
from sklearn.feature_extraction import DictVectorizer

threshold = 5
data_train_0 = data_train.copy()
one_hot = ["Category", "ContractType", "ContractTime"]
embedding = {"LocationNormalized": None, "Company": None}
assert (data_train[one_hot].nunique() == data[one_hot].nunique()).all()

for col in embedding:
    mask = data_train[col].value_counts() >= threshold
    drop = mask.index[mask.values]
    data_train[col] = data_train[col].where(data_train[col].isin(drop), "NaN")
    embedding[col] = data_train[col].unique()
    
categorical_vectorizer = DictVectorizer(dtype=bool, sparse=False)
categorical_vectorizer.fit(data_train[one_hot].apply(dict, axis=1))

DictVectorizer(dtype=<class 'bool'>, sparse=False)

In [13]:
from torch.utils.data.dataloader import Dataset, DataLoader

cat_to_idx = {col : map_idx(names)
              for col, names in embedding.items() }

UNK_IX, PAD_IX = map(token_to_id.get, [UNK, PAD])

class MyDataset(Dataset):
    def __init__(self, data: pd.DataFrame):
        super().__init__()
        self.title: list[torch.Tensor]
        self.descr: list[torch.Tensor]
        self.cat: torch.Tensor
        self.cat_emb: torch.Tensor
        self.target: torch.Tensor
        
        self.precompute_data(data)
    
    def precompute_data(self, data: pd.DataFrame) -> None:
        self.title = [torch.tensor([token_to_id.get(word, UNK_IX)
                                    for word in seq])
                      for seq in data["Title"].str.split()]
        self.descr = [torch.tensor([token_to_id.get(word, UNK_IX)
                                    for word in seq])
                      for seq in data["FullDescription"].str.split()]
        
        self.cat = torch.tensor(categorical_vectorizer.transform(
            data[categorical_columns].apply(dict, axis=1)
        ))
        
        self.cat_emb = torch.tensor([[mapper.get(cat, UNK_IX)
                                      for cat in data[col]]
                                     for col, mapper in cat_to_idx.items()]).T
        self.target = torch.tensor(data[TARGET_COLUMN].values)
    
    def __len__(self):
        return len(self.title)
    
    def __getitem__(self, idx: int):
        x = {attr : getattr(self, attr)[idx] for attr in ["title", "descr", "cat", "cat_emb"]}
        y = self.target[idx]
        return x, y
    

In [14]:
from torch.utils.data._utils.collate import default_collate

def as_matrix(sequences, max_len=None, min_len=5):
    """ Convert a list of tokens into a matrix with padding """
    max_len = np.clip(max(map(len, sequences)), min_len, max_len)
    
    matrix = torch.full((len(sequences), max_len), PAD_IX, dtype=torch.int32)
    for i, seq in enumerate(sequences):
        matrix[i, :min(len(seq), max_len)] = seq[:max_len]
    
    return matrix

def collate_fn(entries):
    texts = {col : as_matrix([entry[0].pop(col) for entry in entries])
             for col in ["title", "descr"]}
    ret = default_collate(entries)
    ret[0]["cat"] = ret[0]["cat"].to(torch.float32)
    ret[0] |= texts
    return ret

In [32]:
train_dataset = MyDataset(data_train)
val_dataset = MyDataset(data_val)
test_dataset = MyDataset(data_test)

In [33]:
batch_size = 512
config = dict(collate_fn=collate_fn, num_workers=0)
train_loader = DataLoader(train_dataset, batch_size, shuffle=True,**config)
val_loader = DataLoader(val_dataset, batch_size, shuffle=False, **config)
test_loader = DataLoader(test_dataset, batch_size, shuffle=False, **config)

In [56]:
class SalaryPredictor(nn.Module):
    def __init__(self, n_tokens=len(tokens), n_cat_features=len(categorical_vectorizer.vocabulary_),
                 hid_size=64, encoded_length=64):
        super().__init__()
        
        self.encoded_len = 64
        outp_features = hid_size * (3 + len(cat_to_idx))
        self.title_encoder = nn.Sequential(
            nn.Embedding(n_tokens, self.encoded_len),
            Apply(torch.permute, (0, 2, 1)),
            nn.Conv1d(self.encoded_len, hid_size, 3),
            nn.AdaptiveMaxPool1d(1),
            Apply(torch.squeeze, dim=-1)
        )
        self.description_encoder = nn.Sequential(
            nn.Embedding(n_tokens, self.encoded_len),
            Apply(torch.permute, (0, 2, 1)),
            nn.Conv1d(self.encoded_len, hid_size, 5, ),
            nn.AdaptiveMaxPool1d(1),
            Apply(torch.squeeze, dim=-1)
        )
        self.cat_encoder = nn.Sequential(
            nn.Linear(n_cat_features, hid_size, bias=False),
            nn.BatchNorm1d(hid_size),
            nn.ReLU(),
            nn.Dropout(.2),
            nn.Linear(hid_size, hid_size, bias=False)
        )
        
        for cat, names in cat_to_idx.items():
            emb_dim = int(len(names) ** .5)
            embedder = nn.Sequential(
                nn.Embedding(len(names), emb_dim),
                nn.Linear(emb_dim, hid_size, bias=False),
                nn.BatchNorm1d(hid_size),
                nn.ReLU(),
                nn.Dropout(.2),
                nn.Linear(hid_size, hid_size, bias=False)
            )
            setattr(self, f"cat_embedder_{cat}", embedder)
            
        self.output = nn.Sequential(
            nn.BatchNorm1d(outp_features),
            nn.ReLU(),
            nn.Linear(outp_features, outp_features, bias=False),
            nn.BatchNorm1d(outp_features),
            nn.ReLU(),
            nn.Dropout(.2),
            nn.Linear(outp_features, 1),
            Apply(torch.squeeze)
        )
        
    def forward(self, batch: dict[str, torch.TensorType]):
        title_enc = self.title_encoder(batch["title"])
        descr_enc = self.description_encoder(batch["descr"])
        cat_enc = self.cat_encoder(batch["cat"])
        cat_embs = []
        for i, cat in enumerate(cat_to_idx):
            embedder = getattr(self, f"cat_embedder_{cat}")
            target = batch["cat_emb"][:, i]
            cat_embs.append(embedder(target))
        concat = torch.hstack([title_enc, descr_enc, cat_enc, *cat_embs])
        return self.output(concat)
    
    def predict(self, batch):
        return self.forward(batch)

# Model 0

In [61]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = SalaryPredictor().to(device)
criterion = nn.MSELoss(reduction='mean')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)
history = History(drop_query="epoch <= 1")
torch.cuda.empty_cache()

In [62]:
%%reprint

plotter = Plotter(bound_history=history)
stopper = EarlyStopper(model=model, save_path="./storage/hw2_m02.pth", bound_history=history, patience=5)

config = dict(model=model, device=device, loss_fn=criterion, optim=optimizer,
              metrics={"loss" : None,
                       "mse" : mean_metric(lambda y_pred, y_true: ((y_pred - y_true)**2).sum()),
                       "mae" : mean_metric(lambda y_pred, y_true: (y_pred - y_true).abs().sum())})
EPOCHS = 100
for _ in history.range(EPOCHS, stopper, do_tqdm=True):
    tr = loopa(**config, dataloader=train_loader, is_train=True)
    val = loopa(**config, dataloader=val_loader, is_train=False)
    history.push_epoch(tr, val)
    plotter.plot()

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

FigureWidget({
    'data': [],
    'layout': {'annotations': [{'font': {'size': 16},
                                'showarrow': False,
                                'text': 'mae',
                                'x': 0.14444444444444446,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 1.0,
                                'yanchor': 'bottom',
                                'yref': 'paper'},
                               {'font': {'size': 16},
                                'showarrow': False,
                                'text': 'mse',
                                'x': 0.5,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 1.0,
                                'yanchor': 'bottom',
                                'yref': 'paper'},
                               {'font': {'size': 16},
               

KeyboardInterrupt: 

In [46]:
while True:
    print(f"epoch: {epoch}")
    epoch += 1
    model.train()
    for i, batch in tqdm(enumerate(
            iterate_minibatches(data_train, batch_size=BATCH_SIZE, device=device)),
            total=ceil(len(data_train) / BATCH_SIZE)
        ):
        pred = model(batch)
        loss = criterion(pred, batch[TARGET_COLUMN])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    history.push_epoch(get_metrics(model, data_train, device=device),
                       get_metrics(model, data_val, device=device))

    plot_metrics(history, ["mse", "mae"], ["mse", "mae"])
    
    if history.val[-1]["mae"] < min_mae:
        min_mae = history.val[-1]["mae"]
        counter = 0
        torch.save(model.state_dict(), "./01_base.pt")
    else:
        counter += 1
        if counter == 3:
            break

In [25]:
from tqdm.auto import tqdm

def batch_to_device(batch, device):
    return {key : val.to(device) for key, val in batch.items()}

def get_metrics(model, dataset, device):
    squared_error = abs_error = num_samples = 0.0
    model.eval()
    with torch.no_grad():
        for batch in tqdm(dataset):
            batch_pred = model(batch)
            squared_error += torch.sum(torch.square(batch_pred - batch[TARGET_COLUMN]))
            abs_error += torch.sum(torch.abs(batch_pred - batch[TARGET_COLUMN]))
            num_samples += len(batch_pred)
    mse = squared_error.detach().cpu().numpy() / num_samples
    mae = abs_error.detach().cpu().numpy() / num_samples
    return {"mse": mse, "mae": mae}

def bar(model, train, val, criterion, optimizer, history, name):
    epoch = 1
    min_mae = np.inf
    counter = 0

    while True:
        print(f"epoch: {epoch}")
        epoch += 1
        model.train()
        train.reshuffle()
        for batches in tqdm(train):
            optimizer.zero_grad()
            
            for batch in batches:
                pred = model(batch)
                loss = criterion(pred, batch[TARGET_COLUMN]) / train.subs_in_batch
                loss.backward()
            optimizer.step()
            
        history.push_epoch(get_metrics(model, train.flatten(), device=device),
                           get_metrics(model, val.flatten(), device=device))
        

        plot_metrics(history, ["mse", "mae"], ["mse", "mae"])
        
        if history.val[-1]["mae"] < min_mae:
            min_mae = history.val[-1]["mae"]
            counter = 0
            torch.save(model.state_dict(), f"./{name}.pt")
        else:
            counter += 1
            if counter == 3:
                break

In [11]:
# from torch.utils.data import Dataset
# import numpy as np
# from random import shuffle
# from toolz import curry




        
        
# class FlattenSubbatches(Dataset):
#     def __init__(self, batches):
#         super().__init__()
#         self.batches = batches
    
#     def __len__(self):
#         return len(self.batches)
    
#     def __getitem__(self, idx: int):
#         if idx >= len(self):
#             raise IndexError
#         return self.batches[idx]

In [12]:
train_dataset = MyDataset(data_train, 64, 4, device)
val_dataset = MyDataset(data_val, 64, 1, device)

model = SalaryPredictor().to(device)
criterion = nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)
history = History()

In [16]:
bar(model, train_dataset, val_dataset, criterion, optimizer, history, "t_model")

# Improvement 0

In [ ]:
# print(categorical_vectorizer.transform(data_train[categorical_columns].apply(dict, axis=1)).nbytes / 1024 ** 3, "Gb")
# for col in categorical_columns:
#     print(col, len(data_train[col].unique()), data_train[col].unique()[:10])

2.747890867292881

The most obvious first improvement is to get rid of dense float32 one-hot matrices to decrease overhead / store all data on gpu.

From [here](https://developers.googleblog.com/2017/11/introducing-tensorflow-feature-columns.html), embeddings of sizes `(num_of_categories)**.25` can be used

In [10]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer


def get_top_n(X, n):
    top, _ = zip(*Counter(X).most_common(n))
    return top

N = 1000

top_n_ordencoder_template = OrdinalEncoder(
    dtype=np.int32,
    max_categories=N + 1, # all infrequent categories will be transformed into `N`
    handle_unknown="use_encoded_value",
    unknown_value=N
)

simple_cats = ["Category", "ContractType", "ContractTime"]
big_cats = ["Company", "LocationNormalized"]
cat_to_ord = ColumnTransformer([
    ("to_num", OrdinalEncoder(dtype=np.int32), simple_cats),
    ("most_freq_to_num", top_n_ordencoder_template, big_cats)
]).fit(data_train)

_ord_for_simple = cat_to_ord.transformers_[0][1]
vocab_sizes = dict(zip(simple_cats, map(len, _ord_for_simple.categories_)))
vocab_sizes |= dict(zip(big_cats, [N + 1] * 2))
vocab_sizes

{'Category': 29,
 'ContractType': 3,
 'ContractTime': 3,
 'Company': 1001,
 'LocationNormalized': 1001}

In [11]:
from torch.utils.data import Dataset, DataLoader
from dataclasses import dataclass

@dataclass
class SeqConfig:
    pad_idx: int
    unk_idx: int
    key_to_index: dict[str, int]
    min_len: int = 5
    max_len: int = None

def seq_to_tensors(sequencies: list[str], seq_config: SeqConfig, device):
    splitted = map(str.split, sequencies)
    lengths = np.cumsum([0] + map(len, splitted))
    tensor = torch.tensor(
        [seq_config.key_to_index.get(word, seq_config.unk_idx)
         for seq in splitted for word in seq[:seq_config.max_len]],
        device=device
    )
    return [tensor[start : fin] for start, fin in zip(lengths, lengths[1:])]

class MyDataset(Dataset):
    def __init__(self, data: pd.DataFrame, device: str, seq_config: SeqConfig):
        super().__init__()
        
        data_dict = {col : seq_to_tensors(data[col], seq_config, device)
                     for col in ["FullDescription", "Title"]}
        
        data_dict["Categorical"] = torch.tensor(cat_to_ord.transform(data), device=device)
        data_dict[TARGET_COLUMN] = torch.tensor(data[TARGET_COLUMN].values, device=device)
        
        self.data = list(zip(*data_dict.values()))
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx: int):
        return self.data[idx]

def collate_fn(dataset_entries, min_len=5):
    full_descrs, titles, cats, targets = zip(*dataset_entries)
    size = len(dataset_entries)
    device = targets[0].device
    
    cat_raw = torch.vstack(cats)
    cat_raw[:, 1:] += torch.tensor(list(vocab_sizes.values()), device=device)[:-1]

    cat = torch.zeros((size, sum(vocab_sizes.values())), device=device)
    cat[torch.tensor(np.arange(size).repeat(5)), cat_raw.flatten()] = 1.
    
    rets = {"Categorical": cat, TARGET_COLUMN: torch.stack(targets)}
    
    for name, sequences in ("FullDescription", full_descrs), ("Title", titles):
        matrix = torch.nn.utils.rnn.pad_sequence(
            sequences, 
            batch_first=True, padding_value=PAD_IX
        )
        if matrix.shape[1] < min_len:
            matrix = torch.hstack([
                matrix, 
                torch.full((matrix.shape[0], min_len - matrix.shape[1]), PAD_IX, device=device)
            ])
            
        rets[name] = matrix
    
    return rets

def get_dataloader(data, batch_size, config, device=device):
    return DataLoader(
        MyDataset(data, device, config),
        batch_size=batch_size, shuffle=True, collate_fn=collate_fn
    )

In [12]:
from pathlib import Path

def get_metrics(model, dataset):
    squared_error = abs_error = num_samples = 0.0
    model.eval()
    with torch.no_grad():
        for batch in tqdm(dataset):
            batch_pred = model(batch)
            squared_error += torch.sum(torch.square(batch_pred - batch[TARGET_COLUMN]))
            abs_error += torch.sum(torch.abs(batch_pred - batch[TARGET_COLUMN]))
            num_samples += len(batch_pred)
    mse = squared_error.detach().cpu().numpy() / num_samples
    mae = abs_error.detach().cpu().numpy() / num_samples
    return {"mse": mse, "mae": mae}

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def do_model(model, train, val, criterion, optimizer, scheduler, history: History, name, patience = 5):
    epoch = 1
    min_mae = np.inf
    counter = 0

    while True:
        print(f"epoch: {epoch}")
        epoch += 1
        model.train()
        for batch in tqdm(train):
            pred = model(batch)
            loss = criterion(pred, batch[TARGET_COLUMN])
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        history.push_epoch(get_metrics(model, train) | {"lr" : get_lr(optimizer)},
                           get_metrics(model, val))
        scheduler.step(history.val[-1]["mae"])
        
        dir_ = Path("./plots")
        if not dir_.exists():
            dir_.mkdir()
        
        plot_metrics(history, metrics=["mse", "mae", "lr"], path=dir_/(name + ".png"),
                     plot_cols=2, height=1000, custom_range={"lr": (-.1, 1.1)})
        
        if history.val[-1]["mae"] < min_mae:
            min_mae = history.val[-1]["mae"]
            counter = 0
            torch.save(model.state_dict(), f"./{name}.pt")
        else:
            counter += 1
            if counter >= patience:
                break

In [13]:
class SalaryPredictorNoOHE(SalaryPredictor):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.cat_embed = [nn.Embedding(vocab_size, max(2, ceil(vocab_size ** .5)))
                          for vocab_size in vocab_sizes.values()]
        
        for col, embed in zip(vocab_sizes.keys(), self.cat_embed):
            setattr(self, f"{col}_embed", embed)
        
        # change input size
        self.output[0] = nn.LazyBatchNorm1d()
        self.output[1] = nn.LazyLinear(self._hid_size)
        print(128 + sum(vocab_sizes.values()))
    
    def forward(self, batch: dict[str, torch.TensorType]):
        cats = torch.hstack([torch.squeeze(embedder(feature), dim=1) 
                             for embedder, feature in zip(self.cat_embed, batch["Categorical"].T)])
        
        concat = torch.cat((
            self.title_encoder(batch["Title"]),
            self.description_encoder(batch["FullDescription"]),
            cats
        ), axis=1)
        
        return self.output(concat)

In [17]:
train_dataset = get_dataloader(data_train, 256)
val_dataset = get_dataloader(data_val, 512)

In [25]:
model = SalaryPredictor(n_cat_features=sum(vocab_sizes.values())).to(device)
criterion = nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)
history = History()

do_model(model, train_dataset, val_dataset, criterion, optimizer, scheduler, history, "03_simple_full_gpu")

In [ ]:
train_dataset = get_dataloader(data_train, 256)
val_dataset = get_dataloader(data_val, 512)

model = SalaryPredictorNoOHE().to(device)
criterion = nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)
history = History()

do_model(model, train_dataset, val_dataset, criterion, optimizer, history, "02_no_ohe")

epoch: 37


  0%|          | 0/765 [00:00<?, ?it/s]

  0%|          | 0/765 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [14]:
import gensim.downloader as api
embedding_model = api.load('glove-twitter-100')

num_of_words = embedding_model.vectors.shape[0]
config = SeqConfig(pad_idx=num_of_words, unk_idx=num_of_words + 1,
                   key_to_index=embedding_model.key_to_index)

In [15]:
model = SalaryPredictor(n_cat_features=sum(vocab_sizes.values()),
                        encoded_length=embedding_model.vector_size)
embedding_layer = nn.Embedding.from_pretrained(torch.vstack([
    torch.tensor(embedding_model.vectors),
    torch.randn((2, embedding_model.vector_size))
]))
model.title_encoder[0] = model.description_encoder[0] = embedding_layer
model = model.to(device)

In [16]:
train_dataset = get_dataloader(data_train, 256, config)
val_dataset = get_dataloader(data_val, 1024, config)

In [24]:
def debug_dataset(data, config):
    return get_dataloader(data[:100], 10, config, 'cpu')

In [98]:
criterion = nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=.95)
history = History()

do_model(model, train_dataset, val_dataset, criterion, optimizer, scheduler, history, "04.0_transfer")

epoch: 16


  0%|          | 0/765 [00:00<?, ?it/s]

  0%|          | 0/765 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
torch.optim.lr_scheduler.ReduceLROnPlateau?

In [105]:
# for param_group in optimizer.param_groups:
#     param_group['lr'] = 3e-5
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=4)

do_model(model, train_dataset, val_dataset, criterion, optimizer, scheduler, history, "04.0_transfer")

In [17]:
del model
torch.cuda.empty_cache()

In [18]:
model = SalaryPredictor(n_cat_features=sum(vocab_sizes.values()),
                        encoded_length=embedding_model.vector_size)

model.title_encoder[0], model.description_encoder[0] = [
    nn.Embedding.from_pretrained(torch.vstack([
        torch.tensor(embedding_model.vectors),
        torch.randn((2, embedding_model.vector_size))
    ]))
] * 2
model = model.to(device)

criterion = nn.MSELoss(reduction='sum')
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3)
history = History(drop_first=1)


In [19]:
do_model(model, train_dataset, val_dataset, criterion, optimizer, scheduler, history, "04.t_transfer", patience=7)

epoch: 3


  0%|          | 0/765 [00:00<?, ?it/s]

  0%|          | 0/765 [00:00<?, ?it/s]

KeyboardInterrupt: 

# Ideas

### A short report

Please tell us what you did and how did it work.

`<YOUR_TEXT_HERE>`, i guess...

## Recommended options

#### A) CNN architecture

All the tricks you know about dense and convolutional neural networks apply here as well.
* Dropout. Nuff said.
* Batch Norm. This time it's `nn.BatchNorm*`/`L.BatchNormalization`
* Parallel convolution layers. The idea is that you apply several nn.Conv1d to the same embeddings and concatenate output channels.
* More layers, more neurons, ya know...


#### B) Play with pooling

There's more than one way to perform pooling:
* Max over time (independently for each feature)
* Average over time (excluding PAD)
* Softmax-pooling:
$$ out_{i, t} = \sum_t {h_{i,t} \cdot {{e ^ {h_{i, t}}} \over \sum_\tau e ^ {h_{j, \tau}} } }$$

* Attentive pooling
$$ out_{i, t} = \sum_t {h_{i,t} \cdot Attn(h_t)}$$

, where $$ Attn(h_t) = {{e ^ {NN_{attn}(h_t)}} \over \sum_\tau e ^ {NN_{attn}(h_\tau)}}  $$
and $NN_{attn}$ is a dense layer.

The optimal score is usually achieved by concatenating several different poolings, including several attentive pooling with different $NN_{attn}$ (aka multi-headed attention).

The catch is that keras layers do not inlude those toys. You will have to [write your own keras layer](https://keras.io/layers/writing-your-own-keras-layers/). Or use pure tensorflow, it might even be easier :)

#### C) Fun with words

It's not always a good idea to train embeddings from scratch. Here's a few tricks:

* Use a pre-trained embeddings from `gensim.downloader.load`. See last lecture.
* Start with pre-trained embeddings, then fine-tune them with gradient descent. You may or may not download pre-trained embeddings from [here](http://nlp.stanford.edu/data/glove.6B.zip) and follow this [manual](https://keras.io/examples/nlp/pretrained_word_embeddings/) to initialize your Keras embedding layer with downloaded weights.
* Use the same embedding matrix in title and desc vectorizer


#### D) Going recurrent

We've already learned that recurrent networks can do cool stuff in sequence modelling. Turns out, they're not useless for classification as well. With some tricks of course..

* Like convolutional layers, LSTM should be pooled into a fixed-size vector with some of the poolings.
* Since you know all the text in advance, use bidirectional RNN
  * Run one LSTM from left to right
  * Run another in parallel from right to left 
  * Concatenate their output sequences along unit axis (dim=-1)

* It might be good idea to mix convolutions and recurrent layers differently for title and description


#### E) <s>Optimizing seriously</s>

* You don't necessarily need 100 epochs. Use early stopping. If you've never done this before, take a look at [early stopping callback(keras)](https://keras.io/callbacks/#earlystopping) or in [pytorch(lightning)](https://pytorch-lightning.readthedocs.io/en/latest/common/early_stopping.html).
  * In short, train until you notice that validation
  * Maintain the best-on-validation snapshot via `model.save(file_name)`
  * Plotting learning curves is usually a good idea
  
Good luck! And may the force be with you!